In [1]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import matplotlib
import plotly.graph_objs as go
import plotly
import os
import seaborn as sns
from plotly.graph_objs import *
from ipywidgets import Image, HTML, Layout, Output, VBox, HBox
from IPython.display import Audio, display

In [2]:
import io
import librosa
import librosa.display
import matplotlib.pyplot as plt

In [3]:
df = pd.read_pickle('spec_df.pkl')

In [4]:
df

,level_0,index,Name,Start,Duration,Time Format,Type,Description,start_s,duration_s,...,UMAP_DTW_1,UMAP_DTW_2,UMAP_DTW_3,UMAP_DTW_4,UMAP_DTW_5,UMAP_1,UMAP_2,UMAP_3,UMAP_4,UMAP_5
0,3335,21,chitter,0:35.655,0:00.044,decimal,Cue,NaN,35.655,0.044,...,9.543516,4.690871,1.973952,3.652256,4.160063,7.290050,0.373394,6.881103,3.297683,0.903807
1,7515,39,chitter,0:34.365,0:00.049,decimal,Cue,NaN,34.365,0.049,...,10.171698,3.432665,1.390636,3.532198,4.517011,7.942714,2.984524,6.364526,4.475345,2.916527
2,2894,25,chitter,0:45.650,0:00.017,decimal,Cue,NaN,45.650,0.017,...,10.796820,3.511225,2.714579,3.440598,3.980539,9.097537,0.672109,7.282919,4.699615,1.395363
3,3810,168,chitter,1:33.687,0:00.054,decimal,Cue,NaN,93.687,0.054,...,10.692395,4.251935,0.830744,3.715116,4.625848,7.081136,3.070330,6.863920,4.631246,3.697712
4,816,44,chitter,0:31.220,0:00.048,decimal,Cue,NaN,31.220,0.048,...,9.548499,4.564956,1.597153,3.496691,4.316077,6.698642,2.432062,6.336713,3.644416,2.599165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1376,7602,126,chirpgr,1:20.869,0:00.081,decimal,Cue,NaN,80.869,0.081,...,12.173289,7.367636,2.859567,2.515017,3.327786,4.673812,2.091206,7.533844,1.833529,2.927159
1377,7603,127,chirpgr,1:21.076,0:00.085,decimal,Cue,NaN,81.076,0.085,...,12.135625,7.384192,2.891701,2.505248,3.323706,4.686907,2.101116,7.514527,1.847097,2.924037
1378,7604,128,chirpgr,1:21.286,0:00.055,decimal,Cue,NaN,81.286,0.055,...,11.173288,4.037132,2.348610,3.236720,3.917331,8.633692,0.856049,7.338339,3.762555,1.453660
1379,7605,129,chirpgr,1:21.440,0:00.092,decimal,Cue,NaN,81.440,0.092,...,12.178346,7.364241,2.844322,2.522502,3.337998,4.718948,2.168444,7.459273,1.870318,2.964444


In [5]:
SR = 48000
FFT_WIN = 0.03 # FFT_WIN*samplerate = length of fft/n_fft (number of audio frames that go in one fft)
FFT_HOP = FFT_WIN/8 # FFT_HOP*samplerate = n of audio frames between successive ffts
FMAX = SR/2

In [8]:
# fmin and fmax are different for preprocessed specs!
librosa.mel_frequencies(n_mels=40, fmin=0.0, fmax=FMAX, htk=False)

array([    0.        ,   104.65819224,   209.31638447,   313.97457671,
         418.63276894,   523.29096118,   627.94915342,   732.60734565,
         837.26553789,   941.92373012,  1049.21141573,  1168.79160686,
        1302.00053085,  1450.39147474,  1615.694756  ,  1799.83789895,
        2004.96811077,  2233.477319  ,  2488.03006278,  2771.59456271,
        3087.47733196,  3439.36173191,  3831.35092214,  4268.01570548,
        4754.44782596,  5296.31934126,  5899.94876196,  6572.37472873,
        7321.43808661,  8155.87331345,  9085.41036859, 10120.88814935,
       11274.38087835, 12559.3388954 , 13990.74549561, 15585.29164258,
       17361.57059398, 19340.29470877, 21544.5369644 , 24000.        ])

In [12]:
# preprocessed specs

FMIN = 620
FMAX = 14000

image_data = {}

for i,dat in enumerate(df.denoised_spectrograms):
    print('\rProcessing i:',i, end='')
    dat = np.asarray(df.iloc[i]['preprocessed_specs'])       
    plt.figure()
    librosa.display.specshow(dat,sr=SR, hop_length=int(FFT_HOP * SR) , fmin=FMIN, fmax=FMAX, y_axis='mel', x_axis='s',cmap='inferno')
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    byte_im = buf.getvalue()
    image_data[df.iloc[i]['callID']] = byte_im
    plt.close()

# Store data (serialize)
with open('image_data.pickle', 'wb') as handle:
    pickle.dump(image_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [6]:
# original specs

FMIN = 0
FMAX = 24000

image_data = {}

for i,dat in enumerate(df.denoised_spectrograms):
    print('\rProcessing i:',i, end='')
    #dat = np.asarray(df.iloc[i]['preprocessed_specs'])    
    dat = np.asarray(df.iloc[i]['spectrograms'])    
    plt.figure()
    librosa.display.specshow(dat,sr=SR, hop_length=int(FFT_HOP * SR) , fmin=FMIN, fmax=FMAX, y_axis='mel', x_axis='s',cmap='inferno')
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    byte_im = buf.getvalue()
    image_data[df.iloc[i]['callID']] = byte_im
    plt.close()

# Store data (serialize)
with open('image_data_ori.pickle', 'wb') as handle:
    pickle.dump(image_data, handle, protocol=pickle.HIGHEST_PROTOCOL)

Processing i: 1

/Users/marathomas/opt/anaconda3/envs/coati_feeding_env/lib/python3.7/site-packages/librosa/display.py:862: MatplotlibDeprecationWarning: The 'basey' parameter of __init__() has been renamed 'base' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)
/Users/marathomas/opt/anaconda3/envs/coati_feeding_env/lib/python3.7/site-packages/librosa/display.py:862: MatplotlibDeprecationWarning: The 'linthreshy' parameter of __init__() has been renamed 'linthresh' since Matplotlib 3.3; support for the old name will be dropped two minor releases later.
  scaler(mode, **kwargs)


Processing i: 1321

In [9]:
%pwd

'/Users/marathomas/Documents/MPI_work/coati_feeding'